# Intro 
Supervised Learning

Date Range: Sep 2021 to March 2022

Linear regression
- Target    = Recovery cases
- Feature   = date, vaccination, case_new, case_active, case_unvax, case_fvax

KNN regression
- Target    = Recovery cases
- Feature   = date, vaccination, case_new, case_active, case_unvax, case_fvax


# Prerequisite

In [7]:
import random
import time
import csv

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# TensorFlow
import tensorflow as tf
from tensorflow.keras import layers, models

# Loading DataSet from Github

In [8]:
# URL should be https://raw.githubusercontent.com/username/repository/branch/filename.csv so no download needed

# Cases & Info: https://github.com/MoH-Malaysia/covid19-public/tree/main/epidemic/{specific_file}
# Vaccination : https://github.com/CITF-Malaysia/citf-public/tree/main/vaccination/{specific_file}

url_cases = "https://raw.githubusercontent.com/MoH-Malaysia/covid19-public/main/epidemic/cases_malaysia.csv"
url_vax = "https://raw.githubusercontent.com/CITF-Malaysia/citf-public/main/vaccination/vax_malaysia.csv"


In [17]:
# Linear Regression
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

columnsCases = ['date', 'cases_new', 'cases_active', 'cases_unvax', 'cases_fvax', 'cases_recovered']
columnsVax = ['date', 'daily']

df_cases = pd.read_csv(url_cases, usecols=columnsCases)
df_vax = pd.read_csv(url_vax, usecols=columnsVax)

start = '2021-09-01'
end = '2022-03-31'

df_cases = df_cases[(df_cases['date'] >= start) & (df_cases['date'] <= end)]
df_vax = df_vax[(df_vax['date'] >= start) & (df_vax['date'] <= end)]

print(df_cases.to_string(index=False))
print(df_vax.to_string(index=False))

      date  cases_new  cases_recovered  cases_active  cases_unvax  cases_fvax
2021-09-01      18762            21073        265279         8299        5487
2021-09-02      20988            23473        262545         9557        6263
2021-09-03      19378            22399        259194         8598        6015
2021-09-04      19057            21582        256307         8308        6213
2021-09-05      20396            20573        255794         8852        6875
2021-09-06      17352            20201        252673         7306        5969
2021-09-07      18547            18902        252007         8142        6134
2021-09-08      19733            22701        248678         8191        7017
2021-09-09      19307            24855        242808         8020        6930
2021-09-10      21176            21476        242167         8437        7964
2021-09-11      19550            21771        239354         8187        7299
2021-09-12      19198            20980        237280         747